In [1]:
%%javascript
if(IPython.tab_as_tab_everywhere)IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import os, sys, json, hashlib
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.metrics import roc_auc_score
from tensorflow.python.keras.models import load_model

/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
test = pd.read_csv('data/application_test.csv', encoding='ISO-8859-1')

In [4]:
model = load_model('credit.keras')
clean = json.load(open('credit.json'))

In [5]:
clean['preprocess']

{'AMT_ANNUITY': {'noNaN': 0, 'type': 'num'},
 'AMT_CREDIT': {'noNaN': 1, 'type': 'num'},
 'AMT_GOODS_PRICE': {'noNaN': 0, 'type': 'num'},
 'AMT_INCOME_TOTAL': {'noNaN': 1, 'type': 'num'},
 'AMT_REQ_CREDIT_BUREAU_DAY': {'noNaN': 0, 'type': 'num'},
 'AMT_REQ_CREDIT_BUREAU_HOUR': {'noNaN': 0, 'type': 'num'},
 'AMT_REQ_CREDIT_BUREAU_MON': {'noNaN': 0, 'type': 'num'},
 'AMT_REQ_CREDIT_BUREAU_QRT': {'noNaN': 0, 'type': 'num'},
 'AMT_REQ_CREDIT_BUREAU_WEEK': {'noNaN': 0, 'type': 'num'},
 'AMT_REQ_CREDIT_BUREAU_YEAR': {'noNaN': 0, 'type': 'num'},
 'APARTMENTS_AVG': {'noNaN': 0, 'type': 'num'},
 'APARTMENTS_MEDI': {'noNaN': 0, 'type': 'num'},
 'APARTMENTS_MODE': {'noNaN': 0, 'type': 'num'},
 'BASEMENTAREA_AVG': {'noNaN': 0, 'type': 'num'},
 'BASEMENTAREA_MEDI': {'noNaN': 0, 'type': 'num'},
 'BASEMENTAREA_MODE': {'noNaN': 0, 'type': 'num'},
 'CNT_FAM_MEMBERS': {'noNaN': 0, 'type': 'num'},
 'CODE_GENDER': {'dic': {'F': 0.06999328222555916,
   'M': 0.1014192025433328,
   'XNA': 0.0},
  'type': 'di

In [6]:
df = pd.DataFrame()
test.loc[test['DAYS_EMPLOYED'] == 365243,'DAYS_EMPLOYED'] = 0
df['DAYS_EMPLOYED'] = -test['DAYS_EMPLOYED']
df['DAYS_BIRTH'] = -test['DAYS_BIRTH']
df['DAYS_REGISTRATION'] = -test['DAYS_REGISTRATION']


df['CNT_CHILDREN'] = test['CNT_CHILDREN'].clip(0,4)
df['DAYS_BIRTH'] = df['DAYS_BIRTH']
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED']
df['DAYS_REGISTRATION'] = df['DAYS_REGISTRATION']

#documents
df['documents'] = 1
for id_ in range(2,22):
	df['documents'] += test['FLAG_DOCUMENT_' + str(id_)]


for key, col in clean['preprocess'].items():
	if col['type'] == 'num':
		df[key] = test[key].fillna(0)
		if not col['noNaN']:
			df[key + '_nan'] = test[key].isna()
	elif col['type'] == 'dic':
		df[key] = test[key].fillna('none').map(col['dic'])

In [8]:

df_use = df[clean['use']]
df_last = pd.DataFrame()

for key in df_use:
	df_last[key] = ((df_use[key] - clean['scale'][key]['a']) / clean['scale'][key]['b']).clip(0.0,1.0).fillna(0)

In [9]:
matrix = df_last.astype(float).as_matrix()

In [10]:
predictions = model.predict(matrix)

In [11]:
out = pd.DataFrame()

out['SK_ID_CURR'] = test['SK_ID_CURR']
out['TARGET'] = predictions[:,0]

out.to_csv('predict.csv',index=False)